In [1]:
import matplotlib.pyplot as plt
import numpy as np

import optical_gating_analysis as OG
from tqdm import tqdm

In [2]:
oog = OG.BasicOpticalGating().default()
#oog.sequence_manager.set_source(r"D:\Data\both 800fps\brightfield\*tif")
#oog.settings["include_reference_frames"] = False
oog.run()

Setting source to D:\Data\2012-06-20 13.34.11 vid 2x2 multi phase single plane\brightfield\*tif
Loading reference sequence from D:\Data\2012-06-20 13.34.11 vid 2x2 multi phase single plane\ref_seq.tif


Getting phases: 100%|██████████| 1106/1106 [00:00<00:00, 109449.33it/s]


In [17]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pgf import FigureCanvasPgf

matplotlib.backend_bases.register_backend("pgf", FigureCanvasPgf)
matplotlib.rcParams.update(
    {
        # Adjust to your LaTex-Engine
        "pgf.texsystem": "xelatex",
        "font.family": "serif",
        "text.usetex": True,
        "pgf.rcfonts": False,
        "axes.unicode_minus": False,
    }
)

matplotlib.rcParams['figure.figsize'] = (4.01407, 3.09654)

In [4]:
i_prev = 0
beat_indices = []
for i in range(1, oog.phases.shape[0]):
    if (oog.phases[i] - oog.phases[i - 1]) < -np.pi:
        if abs(i_prev - i) > 5:
            beat_indices.append(i)
            i_prev = i

gradients = []
from scipy.optimize import curve_fit
for i in range(len(beat_indices) - 2):
    xs = range(beat_indices[i], beat_indices[i+2])
    ys = oog.unwrapped_phases[beat_indices[i]:beat_indices[i+2]]
    popt, popc = curve_fit(lambda x, a, b: a * x + b, xs, ys)
    gradients.append(popt[0])

gradients = np.array(OG.radsperframe_to_bps(np.array(gradients),80))


In [26]:
# Gets the noise spectrum by looking at the standard deviation of the change in delta phases over some time period.
def get_noise_spectrum(delta_phases, averaging_window):
    time_windows = range(1, (delta_phases.shape[0] - averaging_window) // 2, 1)
    avg_delta_phases_std = []
    for i, time_window in enumerate(tqdm(time_windows)):
        avg_delta_phases = []
        for i in np.arange(delta_phases.shape[0] - time_window - averaging_window):
            average_delta_phase_init = np.mean(delta_phases[i:i + averaging_window])
            average_delta_phase_final = np.mean(delta_phases[i + time_window:i + time_window + averaging_window])

            est_avg_delta_phases = average_delta_phase_init - average_delta_phase_final

            avg_delta_phases.append(est_avg_delta_phases)

        if time_window == 100:
            plt.figure()
            plt.hist(avg_delta_phases, bins = 50, rasterized = True)
            plt.xlabel("Average delta phase difference (radians)")
            plt.ylabel("Frequency")
            plt.tight_layout()
            plt.savefig("noise_spectrum_hist.pgf")

        avg_delta_phases_std.append(np.std(avg_delta_phases))

    return time_windows, avg_delta_phases_std

In [27]:
dp_noise_spectrum = get_noise_spectrum(oog.delta_phases, oog.sequence_manager.reference_sequence.shape[0])
gr_noise_spectrum = get_noise_spectrum(gradients, 3)

100%|██████████| 12/12 [00:00<00:00, 3001.65it/s]


In [28]:
plt.figure()
plt.plot(*dp_noise_spectrum)
plt.xlabel("Time windows (frames)")
plt.ylabel("Noise spectrum")
plt.tight_layout()
plt.savefig("dp_noise_spectrum_80.pgf")

In [ ]:
plt.figure()
plt.plot(*gr_noise_spectrum)
plt.xlabel("Time windows (frames)")
plt.ylabel("Noise spectrum")
plt.tight_layout()
plt.savefig("gr_noise_spectrum_80.pgf")